#### Inserción de una marca de agua en una imagen reducida
#### Script: watermark.py

**Gabriel Ruiz Martinez**

[ORCID](https://orcid.org/0000-0001-6651-7836) | [Scopus](https://www.scopus.com/authid/detail.uri?authorId=57188985692) | [Posgrado IMTA](http://posgrado.imta.edu.mx/index.php/2-inicio/168-semblanza-curricular-dr-gabriel-ruiz-martinez)

Tecnólogo del Agua | Subcoordinación de Aguas Superficiales y Océanicas.

[Instituto Mexicano de Tecnología del Agua](https://www.gob.mx/imta).
Octubre del 2023


El script tiene las siguientes funcionalidades: 
1) inserta una marca de agua en una image jpg. La marca de agua contiene datos relacionados con coordenadas UTM y fechas. Las coordenadas se importan de una hoja de Excel.
2) Cambia las dimensiones de las imagen a 8 x 6 cms.
3) La imagen redimensionada y con la marca de agua se exporta a una imagen png.

Esta rutina requiere que se encuentre instalado el modulo de
PILLOW en Python.

Created on Sat Oct 7 11:18:49 2023

@author: Gabriel Ruiz


### Importación de módulos

In [3]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

### Entradas del programa

Nombre del archivo de Excel con la información que se agregará en la marca de agua.

In [4]:
fileExcel = 'Puntos_utm.xlsx'

Dimensiones en las cuales se reducirá la imagen original (en cms).

In [5]:
largo = 8
ancho = 6

Coordenadas en la imagen, donde se ubicará la esquina superior izquierda del cuadro de la marca de agua (en pixeles).

In [6]:
x = 260
y = 155

# x = 800
# y = 570

Ruta de acceso donde se encuentra la fuente tipográfica que se usará en la marca de agua.

In [7]:
pathFont = 'C:\\Windows\\Fonts\\Montserrat\\Montserrat-Bold.ttf'

### Codificación

Formando la tupla con la ubicación de la marca de agua.

In [8]:
wpos = tuple([x, y])

Obteniendo el valor de los pixeles según las dimensiones que se quiere reducir la imagen en cm.

In [9]:
tam = [int(np.floor(largo*(96/2.54))), 
       int(np.floor(ancho*(96/2.54)))]

Importando los datos de la hoja de Excel.

In [10]:
try:
    datos = pd.read_excel(fileExcel,
                      header=0,
                      usecols=[1, 2, 3, 4, 5, 6],
                      parse_dates=True,            
                      engine='openpyxl')
except OSError:
    print('Hay un problema con el archivo de Excel, no se pudo cargar')

In [ ]:
for f in os.listdir(os.getcwd()):
    if f.endswith('.jpg') and f[0] == 'p':
        
        print('Procesando archivo: {:s}'.format(f))
        
        # Conociendo el punto y no de foto analizados
        noPoint = f[2:5]
        noFoto = f[6:8]
        
        # Encontrando el numero de indice del Dataframe que le corresponde 
        # al punto
        try:
            ptoIndex = datos.index[datos['No'] == int(noPoint)][0]
            
            # Importando la imagen a Python, reduciendo sus dimensiones
            try:
                # Abriendo la imagen
                with Image.open(f) as im:
                    #im.show()
                    im.thumbnail(tam, Image.Resampling.LANCZOS)
                    #im.save('a.jpg','JPEG')
                    plt.imshow(im)
                # Creando un copia de la imagen, la copia sera la 
                # imagen que vamos a editar
                imaEdit = im.copy()
            
                # Creando un objeto que se usara para dibujar en la imagen
                imagen = ImageDraw.Draw(imaEdit)

                # Estableciendo la fuente de la marca de agua
                tipo = ImageFont.truetype(font=pathFont, 
                                          size=8, 
                                          encoding='unic')
            
                # Datos de la marca de agua
                xutm = str(np.floor(datos.iloc[ptoIndex][0]*100)/100)
                yutm = str(np.floor(datos.iloc[ptoIndex][1]*100)/100)
                pto = str(int(datos.iloc[ptoIndex][2]))
                dia = str(datos.iloc[ptoIndex][3].day)
                mes = str(datos.iloc[ptoIndex][3].month)
                year = str(datos.iloc[ptoIndex][3].year)
            
                if datos.iloc[ptoIndex][4].hour <= 9:
                    hora = '0' + str(datos.iloc[ptoIndex][4].hour)
                else:
                    hora = str(datos.iloc[ptoIndex][4].hour)
                if datos.iloc[ptoIndex][4].minute <= 9:
                    minuto = '0' + str(datos.iloc[ptoIndex][4].minute)
                else:
                    minuto = str(datos.iloc[ptoIndex][4].minute)
                
                frente = datos.iloc[ptoIndex][5]
            
                # Texto de la marca de agua
                marca_agua = 'Tramo 7\n'+'X = ' + xutm + '\n' + 'Y = ' + yutm+\
                         '\n' + 'Observacion P:' + pto + '\n' + dia + '/' + \
                          mes + '/' + year + ' ' + hora + ":" + minuto + '\n'+\
                          frente
                          
                # Agregando la marca de agua
                imagen.multiline_text(wpos, 
                                  marca_agua,
                                  fill=None,
                                  font=tipo,
                                  anchor='ms',
                                  spacing=0,
                                  align='left',
                                  stroke_fill='black',
                                  stroke_width=1)
                
                # Exportando la imagen con la marca de agua en la imagen reducida
                imaEdit.save('p_' + noPoint + '_' + noFoto + '.png', 'PNG')
                #imaEdit.show()        
                
            except OSError:
                print('Hay un problema al abrir la imagen')
            
        except:
            print('El archivo {:s} es un punto que'.format(f))
            print('no se encuentra en la base de datos!')